In [ ]:
# default_exp util

In [ ]:
#hide
from nbdev.showdoc import *

# ProofZero Utility SDK

A feature parser-hasher client to the Proof Zero matching engine.

The Proof Zero 

## Exported types

In [ ]:
#export
from typing import NewType
Hash = NewType('Hash', str)

## Loading data

In [ ]:
#export
import pandas as pd
def sha2(filename: str) -> pd.DataFrame:
    """
    Convenience function for loading files using `pandas`. Full version supports XLSX, etc.
    """
    return pd.read_csv(filename)